# Workflow widgets example

This notebook illustrates how we can use ESSreduce’s [workflow widgets](https://scipp.github.io/essreduce/user-guide/widget.html) to generate a graphical interface for running the LoKI tutorial workflow.

## Initializing the GUI

It is as simple as importing the dream submodule and generating a GUI using `workflow_widget` (the workflow automatically registers itself to a library of workflows when imported).

In [ ]:
# Import dream submodule to register workflow
from ess import dream  # noqa: F401
from ess.reduce import ui

# Prepare a container for accessing the results computed by the GUI
results = {}

# Initialize the GUI widget
widget = ui.workflow_widget(result_registry=results)
widget

## Accessing the results

We can now access the computed result in the `results` dictionary:

In [ ]:
results

The result can be plotted using

In [ ]:
(da,) = results.values()
da.plot(norm="log")